# **长短期记忆LSTM**

## **长短期记忆LSTM**

LSTM中引入了三个门：遗忘门，输入门，输出门，其输入都是上一个时间的隐藏状态$H_{t-1}$和当前时间步的输入$X_t$，输出有sigmoid函数得到    
同时LSTM引入了和隐藏状态相同的记忆细胞，用于记录额外的信息   

### **遗忘门, 输入输出门**

<div align=center>
<img width="500" src="../image/6.8_lstm_0.svg"/>
</div>
<div align=center>图6.7 长短期记忆中输入门、遗忘门和输出门的计算</div>

假设隐藏单元个数为$h$，给定时间步的输入为$X_t \in \mathbb R^{n \times d}$（样本数为n，输入个数为d），和上一个人时间步的输入$H_{t-1} \in \mathbb R^{n \times h}$

那么时间步$t$的输入门$I_t \in \mathbb R^{n \times h}$、遗忘门$F_t \in \mathbb R^{n \times h}$、输出门$O_t \in \mathbb R^{n \times h}$的计算过程为

$\large I_t = \sigma (X_tW_{xi} + H_{t-1}W_{hi} + b_i) \\
 \large F_t = \sigma (X_tW_{xf} + H_{t-1}W_{hf} + b_f) \\ 
 \large O_t = \sigma (X_tW_{xo} + H_{t-1}W_{ho} + b_o)$

### **候选记忆细胞**

接下来，长短期记忆模型需要计算候选记忆细胞$\tilde C_t$,计算方法与3个门类似，但是激活函数是tanh函数

<div align=center>
<img width="500" src="../image/6.8_lstm_1.svg"/>
</div>
<div align=center>图6.8 长短期记忆中候选记忆细胞的计算</div>

$\tilde C_t \in \mathbb R^{b \times h}$具体计算过程下

$\large \tilde C_t = tanh(X_tW_{xc} + H_{t-1}W_{hc} + b_c)$

### **记忆细胞**

我们通过元素值域在$[0, 1]$的遗忘门、输入门来控制隐藏状态的信息流动，一般是用元素乘法实现     
当前时间步的记忆细胞的输出$C_t$的输入是$C_{t-1}$和$\tilde C_t$

$\large C_t = F_t \odot C_{t-1} + I_t \odot \tilde C_t$

所以,**遗忘门控制上个记忆细胞的信息是否能够传入当前时间步**，**输入门控制当前时间步的输入通过候选记忆细胞如何流入当前时间步的记忆细胞**   
这个设计可以应用来处理循环神经网络的梯度衰减问题，并能更好的捕捉较远的依赖

<div align=center>
<img width="500" src="../image/6.8_lstm_2.svg"/>
</div>
<div align=center>图6.9 长短期记忆中记忆细胞的计算</div>

### **隐藏状态**

计算了记忆细胞的输出后，接下来通过输出门控制记忆细胞到隐藏状态的流动：

$\large H_t = O_t \odot tanh(C_t)$

当输出门接近为0时，记忆细胞的信息仅仅自己保留。

<div align=center>
<img width="500" src="../image/6.8_lstm_3.svg"/>
</div>
<div align=center>图6.10 长短期记忆中隐藏状态的计算</div>

## 读取数据集

In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("../utils") 
import d2lzh as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

## **从零实现LSTM**

### **初始化参数**

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

will use cuda


In [8]:
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, dtype=torch.float32, device=device), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, vocab_size))
    b_q = torch.nn.Parameter(torch.zeros(vocab_size, dtype=torch.float32, device=device), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

### **模型构建**

In [9]:
# 初始化隐藏状态，初始化状态有两个一个是记忆细胞一个是隐藏状态
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))

In [10]:
# 定义模型
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * torch.tanh(C)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

### **训练**

In [11]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [12]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 92.954972
 - 分开 我的世界 你们  我们 你不再  你的 再不再  你的灵魂 你不能 你不再再 你不能再开 我不能再
 - 不分开 你的眼界 你不开 你不要再开 你不能再开 我不能再开 我不能再开 我不能再开 我不能再开 我不能再
epoch 80, perplexity 17.010195
 - 分开了 爱在我们起起 你们拳如龙 当敌人是空 我等你无开 我的势空 这么福 是你不出 你的手音 已经了 
 - 不分开 你已经离开 我不能不来 你永远变不住我 不知道你们看着 不想要你说你是是一场悲剧 我想我自己的裁判
epoch 120, perplexity 6.882834
 - 分开了 爱一枝被秋间 我很天上谁 将风远的希悔 你说如开我 不需笑記 我的指变 就是够一种 只快就是 你
 - 不分开 你已经离开了 不知道会想觉 因为我的请生面 用你的时间就微笑着你的脸的脸 雨你在我手手 消人都不开
epoch 160, perplexity 4.037338
 - 分开了 爱中后后下手 让什己会汉 你成手 不不再 也住我也见你的情情 你的城度 我想想得再写 我手的感动
 - 不分开 你永一路我就会有 只是一个梦的感度 有点地黄的味道 这样说好太人爱 没有对你没笑得我错不起 她温色


### pytorch

In [13]:
num_epochs = 80
lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch40, perplexity:1.0594
- 分开 有点讽刺 有点酸性  但它确在这快乐社会发生产生共鸣 产生共鸣来阻止一切暴力  眼泪随着音符吸入血
- 不分开 爱可有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这一种身世 有甚么资格 献世 1983年小巷 1
epoch80, perplexity:1.0247
- 分开 能不能就这样忍着痛泪不掉 说好陪我到老 永恒往哪里找 再次拥抱 一分一秒都好 繁华声 遁入空门 折
- 不分开 你靠在我肩膀 深呼吸怕遗忘 因为捞鱼的蠢游戏我们开始交谈 多希望话题不断园游会永不打烊 气球在我手
